# <center>**TP2**</center>

## **Partie I.** Estimation à noyau : exemple pratique

In [ ]:
# imports

import numpy as np # linear algebra
import pandas as pd # datasets
# libraries and options for plots
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

Pour cet exemple, nous allons considérer un nouveau jeu de données, disponible dans la librairie ``sklearn``. Le dataset en question contient les caractéristiques d'un totale de 178 vins (rouge, rosé et blanc). Nous nous intéressons à la distribution de l'inténsité de la couleur, décrit dans la variable ``color_intensity``.

**1.** Télécharger le dataset, avec la fonction ``load_wine()`` de ``sklearn.datasets``, et visualiser les premières 5 lignes. Ajouter au dataset la colonne `target`, contenant l'information de groupe (rouge, rosé, blanc). 

**2.** Visualiser les données en utilisant un histogramme. Stratifier ensuite à l'aide de la variable `target`.

**3.** Définir l'estimateur de la densité à noyau Gaussien. Dans une même figure, intégrer la densité estimée par histogrammes et par noyau gaussien. Vous pouvez utiliser le même $\nu$.

**4.** Faites varier $\nu$ : qu'est ce que vous observez?

Sous Python, deux librairies existent pour réaliser une estiamtion de la densité à noyau, avec une syntaxe très similaire : [``sklearn.neighbors.KernelDensity``](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KernelDensity.html#sklearn.neighbors.KernelDensity) et [``statsmodels.nonparametric.KDEMultivariate`` et ``statsmodels.nonparametric.KDEUnivariate``](https://www.statsmodels.org/dev/examples/notebooks/generated/kernel_density.html#The-available-kernel-functions). 

Je vous propose d'abord d'utiliser ``statsmodel``, nous verrons ensuite aussi la solution proposée par ``sklearn``.

In [ ]:
%%capture
# The following line needs to be executed just one time, in order to install statsmodel
import sys
!{sys.executable} -m pip install statsmodels

In [ ]:
%%capture
# Ou alors:
!pip install statsmodels

Tout d'abord, nous allons importer la fonctions souhaitée, et lui donner les datas qu'on souhaite modéliser.

In [ ]:
import statsmodels.api as sm
from statsmodels.nonparametric.kde import kernel_switch

kde = sm.nonparametric.KDEUnivariate(data.color_intensity)

Nous pouvons imprimer les noms des kernels proposés. Plus d'info sont disponibles [ici](https://www.statsmodels.org/dev/_modules/statsmodels/nonparametric/kde.html).

In [ ]:
print(kernel_switch.keys())

**5.** Reproduire l'estimation par noyau Gaussien obtenue à l'étape **3.** avec `statsmodels`.

On peut faire la même chose avec ``sklearn``:

In [ ]:
from sklearn.neighbors import KernelDensity

kernel = 'gaussian'

In [ ]:
# estimation de densité par noyaux gaussiens
np_data = data.color_intensity.to_numpy()[:, np.newaxis]
kde = KernelDensity(kernel=kernel, bandwidth=bandwidth).fit(np_data)

In [ ]:
x_plot = np.linspace(0, 14, 1000)[:, np.newaxis]
# calcul de la densité pour les données de X_plot
density = np.exp(kde.score_samples(x_plot))

In [ ]:
fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111)

# Create the plot
ax.plot(x_plot, density, lw=3, label='Kernel function "{}"'.format(kernel))

ax= sns.histplot(data=data,
                 bins=b, 
                 stat='density', # Normalize such that the total area of the histogram equals 1
                 x='color_intensity',
                 # Next arguments are just to have a nice plot
                 color="skyblue",
                 alpha=.3,
                 legend=True)

ax.set(xlabel='Color intensity', ylabel='Density')

plt.grid(True, zorder=-5)

plt.legend(loc='best')

plt.tight_layout()

**6.** Choisissez une librairie, essayez d'autres noyaux. Quel est leur impact sur l'estimation ?

Avec ``sklearn``il est aussi possible de générer des nouvelles données à partir de la densité estimé, pour ensuite les visualiser.

In [ ]:
N=500

x_g = kde.sample(N)
kde2 = KernelDensity(kernel=kernel, bandwidth=bandwidth).fit(x_g)


fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(x_plot, density, 'k-', label="Estimation from observed data")
ax.plot(x_plot, np.exp(kde2.score_samples(x_plot)), 'r-', label="Estimation from newly generated data")
ax= sns.histplot(x_g,
                 bins=b, 
                 stat='density', # Normalize such that the total area of the histogram equals 1
                 # Next arguments are just to have a nice plot
                 alpha=.3)
ax.legend(loc='best')
ax.set(xlabel='Color intensity', ylabel='Density')
plt.show()

Nous pouvons également nous intéresser à deux variables, par exemple l'inténsité de la couleur et l'alcohol, et essayer d'estimer leur densité  jointe.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

In [ ]:
# définir la grille pour la visualisation
grid_size = 100
mx = min(data.color_intensity)
Mx = max(data.color_intensity)
my = min(data.alcohol)
My = max(data.alcohol)
xstep = (Mx - mx) / grid_size
ystep = (My - my) / grid_size
Gx = np.arange(mx, Mx, xstep)
Gy = np.arange(my, My, ystep)
Gx, Gy = np.meshgrid(Gx, Gy)

np_data = np.hstack((data.color_intensity.to_numpy()[:, np.newaxis], data.alcohol.to_numpy()[:, np.newaxis]))

In [ ]:
# définir la largeur de bande pour le noyau
bw = nu

# estimation, puis calcul densité sur la grille
kdet = KernelDensity(kernel='gaussian', bandwidth=bw).fit(np_data)
Z = np.exp(kdet.score_samples(np.hstack((
                    (Gx.reshape(grid_size*grid_size))[:,np.newaxis],
                    (Gy.reshape(grid_size*grid_size)[:,np.newaxis])))))

In [ ]:
# affichage
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(projection='3d')
ax.plot_surface(Gx, Gy, Z.reshape(grid_size,grid_size), rstride=1,
                    cstride=1, cmap=cm.coolwarm, linewidth=0, antialiased=True)

ax.set(xlabel='Color intensity', ylabel='Alcohol', zlabel = "Density")
plt.show()